In [1]:
import torch
import pandas as pd
import numpy as np
import cv2
import os, time, random
import shutil as sh
from tqdm.notebook import tqdm

Link your google drive to colab pro in order to grab the dataset

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

Mounted at /content/gdrive


making a directory on colab machine to store data and results from training

In [3]:
%cd /content/gdrive/MyDrive

/content/gdrive/MyDrive/CAT_Vehicle


In [4]:
%mkdir CarDetection
%cd CarDetection
%mkdir data

mkdir: cannot create directory ‘CarDetection’: File exists
/content/gdrive/MyDrive/CAT_Vehicle/CarDetection
mkdir: cannot create directory ‘data’: File exists


Copying the data over

In [ ]:
%cp -a gdrive/MyDrive/CarDetection/data/. CarDetection/data

cp: cannot stat 'gdrive/MyDrive/CAT_Vehicle/CarDetection/data/.': No such file or directory


In [ ]:
%cd CarDetection/data/data

[Errno 2] No such file or directory: 'CarDetection/data/data'
/content/gdrive/MyDrive/CAT_Vehicle/CarDetection


Unzipping said data, validation and training are already split

In [ ]:
!unzip Train1
!unzip Train2
!unzip Train3
!unzip Train4
!unzip Train5
!unzip Train6
!unzip Train7
!unzip Train8
!unzip Train9
!unzip Train10

In [ ]:
!unzip valid

Archive:  valid.zip
replace valid/image1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


Moving unzipped training data into a singular folder for access

In [ ]:
%mv Train1/* train/
%mv Train2/* train/
%mv Train3/* train/
%mv Train4/* train/
%mv Train5/* train/

In [ ]:
%mv Train6/* train/
%mv Train7/* train/
%mv Train8/* train/
%mv Train9/* train/
%mv Train10/* train/

In [ ]:
%cd ../

/content/CarDetection


Outputting to terminal the number of files in the training data to see if all images and labels are there note: make sure to cd into the training folder, it is not done here

In [5]:
%ls -l | wc -l

2


Clone YOLOv5

In [6]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12270, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
^C


In [ ]:
!which pip

/usr/local/bin/pip


In [ ]:
!pip install -r yolov5/requirements.txt

     |████████████████████████████████| 596 kB 12.4 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%ls

data/  yolov5/


Editing the labels to solely focus on cars and trucks to improve accuracy as we are only planning on localizing these two. However, maybe we should consider keeping all classes to simulate to improve realism, it depends if we are assuming an integrated system design or a "uploading to the cloud" approach.

In [ ]:
# next step lets make the alg focus on the cars and trucks classes and only those
def remove_other_class_labels(path):
    # navigate through the file system

    for filename in os.listdir(path):
        data_keep = []
        if filename.endswith('txt'):
            file_obj = open(path + filename)
            lines = file_obj.readlines()
            for line in lines:
                line = line.strip('\n').split(' ')
                if line[0] == '0' or line[0] == '1':
                    # keep the line
                    line = ' '.join(line) + '\n'
                    data_keep.append(line)
                elif line[0] == '10':
                    # rewrite the line to be line[0] = '2'
                    line[0] = '2'
                    line = ' '.join(line) + '\n'
                    data_keep.append(line)
            file_obj.close()  # close the file to save the contents
            with open(path + filename, 'w') as file_obj:
                file_obj.truncate(0) # remove all contents
                file_obj.writelines(data_keep) # write the correct contents
            file_obj.close()


In [ ]:
train_path = os.getcwd() + '/data/data/train/'
valid_path = os.getcwd() + '/data/data/valid/'
remove_other_class_labels(train_path)
remove_other_class_labels(valid_path)

In [ ]:
%cd yolov5/

/content/CarDetection/yolov5


Training on the dataset

In [ ]:
!python train.py --cfg yolov5s.yaml --batch 80 --epochs 50 --data data.yaml --weights yolov5s.pt --workers 24 --name yolo_road_det


train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=80, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=runs/train, name=yolo_road_det, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-202-g7d46c69 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.01

Moving the results to gdrive as cloud instance will expire a while after training is finished

In [ ]:
%mv yolo_road_det3/ ../../../../gdrive/MyDrive